In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from math import ceil

# Load the labels
df_labels = pd.read_csv("/content/drive/MyDrive/Kury/breeds.csv", sep=";")
train_file = '/content/drive/MyDrive/Kury/train/'

# Number of breeds to use
num_breeds = 5
im_size = 224
batch_size = 64

# Get the top 'num_breeds' breeds
breed_counts = df_labels['breed'].value_counts()
top_breeds = breed_counts.nlargest(num_breeds).index

# Filter the DataFrame to only include images of the top breeds
df_labels = df_labels[df_labels['breed'].isin(top_breeds)]

# Create the image file names
df_labels['img_file'] = df_labels['id'].apply(lambda x: x + ".jpg")

print("Wykorzystane rasy:", df_labels['breed'].unique())

# Encode the breed labels
encoder = LabelEncoder()
df_labels['breed'] = encoder.fit_transform(df_labels['breed'])

# Convert labels to strings to match the requirement for sparse class mode
df_labels['breed'] = df_labels['breed'].astype(str)

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df_labels, test_size=0.2, stratify=df_labels['breed'])

# Data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True, vertical_flip=True, rotation_range=20)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_file,
    x_col='img_file',
    y_col='breed',
    target_size=(im_size, im_size),
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=train_file,
    x_col='img_file',
    y_col='breed',
    target_size=(im_size, im_size),
    batch_size=batch_size,
    class_mode='sparse'
)

# Load the ResNet50V2 model, excluding the top layer, and add custom layers
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(num_breeds, activation='softmax')(x)

# Combine the base model and the custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
epochs = 5
learning_rate = 1e-3
optimizer = RMSprop(learning_rate=learning_rate, rho=0.9)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])

# Calculate steps_per_epoch and validation_steps
steps_per_epoch = ceil(train_generator.samples / batch_size)
validation_steps = ceil(test_generator.samples / batch_size)

# Train the model
hist = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps
)

# Save the model for future predictions
model.save("model")


Wykorzystane rasy: ['leghorn' 'newhampshire' 'plymouthrock' 'silkie' 'sussex']
Found 160 validated image filenames belonging to 5 classes.
Found 40 validated image filenames belonging to 5 classes.
Epoch 1/5
3/3 [==============================] - 45s 15s/step - loss: 0.6440 - accuracy: 0.7875 - val_loss: 0.0270 - val_accuracy: 1.0000
Epoch 2/5
3/3 [==============================] - 37s 10s/step - loss: 0.0793 - accuracy: 0.9688 - val_loss: 0.0109 - val_accuracy: 1.0000
Epoch 3/5
3/3 [==============================] - 37s 14s/step - loss: 0.0221 - accuracy: 0.9875 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 4/5
3/3 [==============================] - 34s 11s/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 39s 13s/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.0048 - val_accuracy: 1.0000
